In [5]:
### Run create_dataset.ipynb first. Ensure you have the train_df saved in {train_df_path}
train_df_path = "/Users/nijiayi/Stats_C161-261_Project/train_data_after_feature_selection.csv"
test_df_path = "/Users/nijiayi/Stats_C161-261_Project/test_data_after_feature_selection.csv"

In [6]:
from load_dataset import separate_labels

df_label_0, df_label_1 = separate_labels(file_path=train_df_path) # Enter the train_df_path
# Remember we should generate label 1 data
# Make sure that df_label_0, df_label_1 are consistent
real_data = df_label_1

4it [00:11,  2.87s/it]


In [3]:
df_label_0.shape, df_label_1.shape

((3766407, 23), (47567, 23))

### Train CTGAN

In [7]:
from sdv.metadata import SingleTableMetadata

metadata = SingleTableMetadata()
metadata.detect_from_dataframe(df_label_1)
for column_name, properties in metadata.columns.items():
    if properties["sdtype"] not in ["numerical", "categorical"]:
        properties["sdtype"] = "categorical"
        properties.pop("pii", None)
metadata

{
    "columns": {
        "city": {
            "sdtype": "categorical"
        },
        "series_dev": {
            "sdtype": "numerical"
        },
        "emui_dev": {
            "sdtype": "numerical"
        },
        "device_name": {
            "sdtype": "numerical"
        },
        "device_size": {
            "sdtype": "numerical"
        },
        "net_type": {
            "sdtype": "categorical"
        },
        "creat_type_cd": {
            "sdtype": "categorical"
        },
        "slot_id": {
            "sdtype": "numerical"
        },
        "spread_app_id": {
            "sdtype": "numerical"
        },
        "app_second_class": {
            "sdtype": "numerical"
        },
        "u_refreshTimes": {
            "sdtype": "categorical"
        },
        "u_feedLifeCycle": {
            "sdtype": "categorical"
        },
        "user_id_count": {
            "sdtype": "numerical"
        },
        "task_id_count": {
            "sdtype": "numerical"


In [ ]:
from sdv.single_table import CTGANSynthesizer
synthesizer = CTGANSynthesizer(metadata, verbose=True)
synthesizer.fit(df_label_1)

In [ ]:
synthesizer.save('ctgan_synthesizer.pkl')

In [ ]:
# If have the synthesizer saved just load it
# import joblib
# synthesizer = joblib.load("tvae_synthesizer.pkl")

In [ ]:
# How many synthetic data we need?
num_generated_data_needed = df_label_0.shape[0] - df_label_1.shape[0]
synthetic_data = synthesizer.sample(num_generated_data_needed)
synthetic_data.to_csv("ctgan_synthesized_data.csv")

### Evaluation